In [1]:
%run ..\notebooks\Util_func.ipynb

In [2]:
# # Bay Area
# REG = 'BayArea'
# base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\Processing_20200228\2_tour_extract\wt_wkday'
# raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\Processing_20200228\spatial_join'

# out_file = r'out\%s_5_TNCTrips_Misc.xlsx' %REG

In [3]:
# # SANDAG
# REG = 'SANDAG'
# base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG\2_tour_extract\wt_wkday'
# raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SANDAG'

# out_file = r'out2\%s_5_TNCTrips_Misc.xlsx' %REG

In [4]:
# SCAG
REG = 'SCAG'
base_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SCAG_dataset_2020-02-27\2_tour_extract\wt_wkday'
raw_dir = r'Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2018\SCAG_dataset_2020-02-27'

out_file = r'out2\%s_5_TNCTrips_Misc.xlsx' %REG

In [5]:
## Process trip records
raw_trips = pd.read_csv(join(raw_dir, 'ex_trip_wZones.csv'))
raw_trips = raw_trips[['hh_id','person_num','trip_num','mode_uber','mode_lyft','tnc_decide','tnc_schedule',
                       'tnc_replace','tnc_pooled','mode_type_imputed']]
raw_trips = raw_trips.rename(columns={'hh_id':'hhno','person_num':'pno','trip_num':'tsvid'})

trip_df = pd.read_csv(join(base_dir,'survey2018_tripx.dat'), sep=' ')
trip_df = link_dt(trip_df)
trip_df = trip_df.loc[trip_df['mode']==9, ]
trip_df = trip_dist_map(trip_df)
trip_df = trip_df.merge(raw_trips, how='left')
trip_df = trip_df[trip_df['mode_type_imputed']!=4] #remove taxi trips

trip_df['tnc_type'] = 3 # prem/other
trip_df.loc[(trip_df['mode_uber']==1) | (trip_df['mode_lyft']==1), 'tnc_type'] = 1 # pooled
trip_df.loc[(trip_df['mode_uber']==2) | (trip_df['mode_lyft']==2), 'tnc_type'] = 2 # regular

trip_df['count'] = 1
trip_df = trip_df[(trip_df['trexpfac']>0) & (trip_df['mode']>0)]
trip_df = trip_df[(trip_df['otaz']>0) & (trip_df['dtaz']>0)]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [6]:
col_dict = {
    'altmode': {
        'desc': 'TNC_Alt',
        'col': 'tnc_replace',
        'vals': list(range(1,8)) + [9,10,997,995],
        'labels': ['1_Taxi','2_DA','3_SR','4_Trn','5_WkBk','6_ODes','7_NoTrp','90_BART','91_OTrn','92_Oth','93_Miss']
    },
    'when': {
        'desc': 'TNC_When',
        'col': 'tnc_decide',
        'vals': list(range(1,6)) + [995],
        'labels': ['1_RightBef','2_Hour','3_SameDay','4_PrevDay','5_2+Days','6_Miss']
    },
    'sched': {
        'desc': 'TNC_Sched',
        'col': 'tnc_schedule',
        'vals': list(range(2)) + [995],
        'labels': ['1_No','2_Yes','3_Miss']
    },
    'tncmode': {
        'desc': 'TNC_Mode',
        'col': 'tnc_type',
        'vals': range(1,4),
        'labels': ['2_Pool','1_Reg','3_PremOth']
    }
}

In [7]:
fname = out_file
writer = pd.ExcelWriter(fname, engine='xlsxwriter')
workbook = writer.book
format1 = workbook.add_format({'num_format': '#,##0.0'})

In [8]:
from xlsxwriter.utility import xl_rowcol_to_cell

for key in ['when', 'sched', 'altmode', 'tncmode']:
    d1_dict = col_dict[key]
    row = 0
    sname = d1_dict['desc']
    
    title = 'TNC Trips by ' + d1_dict['desc']
    tab = prep_data_1d(trip_df, d1_dict['desc'],d1_dict['col'], 'trexpfac', d1_dict['vals'], d1_dict['labels'])
    row = write_to_excel(tab, sname, title, row)

    title = 'Column Shares by ' + d1_dict['desc']
    row = write_to_excel(getSharesIdx(tab.copy()), sname, title, row)

    tab2 = tab.copy()
    tab2.iloc[-1,-1] = tab2.iloc[-1,0]

    title = 'Column Shares 95% CI by ' + d1_dict['desc']
    row = write_to_excel(getSharesIdxCI95(tab.copy()), sname, title, row)

    tab_range = xl_rowcol_to_cell(row,1) + ':' + xl_rowcol_to_cell(row,tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [9]:
writer.save()